In [1]:
from google.colab import drive
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Mount Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
train_dir = '/content/drive/My Drive/Subhan Model Data/'


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

# Step 1: Data Preprocessing & Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Rescaling the pixel values to the range [0, 1]
    rotation_range=30,  # Random rotation in the range of 30 degrees
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Random shear
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Fill in pixels with the nearest valid value
)


In [4]:
# Step 2: Load the images from the directory and assign labels based on the folder structure
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize all images to 224x224
    batch_size=32,
    class_mode='categorical',  # As we have multiple categories (combining uniform and batch card status)
    shuffle=True
)


Found 948 images belonging to 3 classes.


In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.keras.applications import MobileNetV2

# Define the image size
IMG_SIZE = 224  # You can change this to 256 or other sizes based on your requirements

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers
base_model.trainable = False

# Build custom model
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = Flatten()(x)
x = Dropout(0.5)(x)

# Student output (3 categories)
student_output = Dense(3, activation='softmax', name='student_output')(x)

# Uniform output (2 categories: Batch card vs no batch card)
uniform_output = Dense(2, activation='softmax', name='uniform_output')(x)

# Define the model
model = Model(inputs, [student_output, uniform_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={'student_output': 'categorical_crossentropy', 'uniform_output': 'categorical_crossentropy'},
    metrics={'student_output': 'accuracy', 'uniform_output': 'accuracy'}
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ mobilenetv2_1.00_224      │ (None, 7, 7, 1280)     │      2,257,984 │ input_layer_1[0][0]    │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 62720)          │              0 │ mobilenetv2_1.00_224[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 62720)          │              0 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ student_output (Dense)    │ (None, 3)              │        188,163 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ uniform_output (Dense)    │ (None, 2)              │        125,442 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,571,589 (9.81 MB)

 Trainable params: 313,605 (1.20 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# Step 5: Set up a callback to save the best model during training
checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Subhan Project/testData/student_uniform_classifier_v4.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)


In [9]:
# Step 6: Train the Model
history = model.fit(
    train_generator,
    epochs=5,  # You can increase the number of epochs based on your requirement
    callbacks=[checkpoint]
)

Epoch 1/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 207s 5s/step - loss: 8.3737 - student_output_accuracy: 0.5580
Epoch 2/5


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - loss: 1.0105 - student_output_accuracy: 0.8917
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - loss: 0.7438 - student_output_accuracy: 0.9002
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - loss: 0.3731 - student_output_accuracy: 0.9532
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - loss: 0.1563 - student_output_accuracy: 0.9679


In [13]:
# Print the keys of the history object to check what metrics are available
print(history.history.keys())


dict_keys(['loss', 'student_output_accuracy'])


In [17]:
from google.colab import files
model.save('trained_model.h5')  # Replace 'trained_model.h5' with your desired filename
files.download('trained_model.h5')  # Replace with your model's filename


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>